In [ ]:
import wandb
import pandas as pd
import plotly.express as px

In [ ]:
# Initialize wandb API
api = wandb.Api()

# Specify your entity and project name
entity = ""  # Replace with your wandb username or team name
project = ""  # Replace with your project name

# Get all runs from the project
runs = api.runs(f"{entity}/{project}")


In [ ]:

# Fetch data from all runs
all_data = []
for run in runs:
    run_data = {
        'name': run.name,
        'id': run.id,
        'state': run.state,
        'created_at': run.created_at,
        'config': run.config,
        'summary': run.summary._json_dict,
        'history': run.history().to_dict('records')
    }
    all_data.append(run_data)

print(f"Fetched data from {len(all_data)} runs")

df = pd.DataFrame(all_data)

In [ ]:
df.to_csv("wandb_runs_data.csv", index=False)

In [ ]:
df = pd.read_csv("wandb_runs_data.csv")

In [ ]:
df

In [ ]:
run_name_1 = ""
run_name_2 = ""

In [ ]:
def get_run_row(df, run_name):
    return df[df['name'] == run_name].iloc[0]

In [ ]:
run_1 = get_run_row(df, run_name_1)
run_2 = get_run_row(df, run_name_2)

In [ ]:
eval(run_row.config)

In [ ]:
eval(run_1.summary)

In [ ]:
run_1.history

In [ ]:
pd.DataFrame(df.iloc[1]['history'])

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import ast

# Create a dropdown widget for selecting run
run_dropdown = widgets.Dropdown(
    options=[(f"{row['name']} (id: {row['id']})", idx) for idx, row in df.iterrows()],
    value=0,
    description='Select Run:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Create output widget for the plot
output = widgets.Output()

def update_plot(change):
    with output:
        output.clear_output(wait=True)
        
        # Get history for selected run
        history_data = df.iloc[change['new']]['history']
        print(history_data)
        
        
        hist = pd.DataFrame(history_data)
        
        # Get available numeric columns from the history dataframe
        numeric_columns = hist.select_dtypes(include=['float64', 'int64']).columns.tolist()
        # Remove timestamp and step columns as they're typically x-axis values
        plot_columns = [col for col in numeric_columns if col not in ['_timestamp', '_step', 'train/global_step']]
        
        # Filter out columns that are all NaN
        plot_columns = [col for col in plot_columns if hist[col].notna().any()]
        
        if not plot_columns:
            print("No plottable data found")
            return
        
        # Create figure with dropdown menu
        fig = go.Figure()
        
        # Add traces for each column (initially all hidden except the first)
        for i, col in enumerate(plot_columns):
            fig.add_trace(
                go.Scatter(
                    x=hist.index,
                    y=hist[col],
                    name=col,
                    visible=(i == 0)  # Only first trace is visible initially
                )
            )
        
        # Create dropdown menu buttons
        buttons = []
        for i, col in enumerate(plot_columns):
            visible = [False] * len(plot_columns)
            visible[i] = True
            buttons.append(
                dict(
                    label=col,
                    method="update",
                    args=[{"visible": visible},
                          {"title": f"{df.iloc[change['new']]['name']} - {col}"}]
                )
            )
        
        # Add dropdown menu
        fig.update_layout(
            updatemenus=[
                dict(
                    active=0,
                    buttons=buttons,
                    direction="down",
                    showactive=True,
                    x=0.1,
                    xanchor="left",
                    y=1.15,
                    yanchor="top"
                )
            ],
            title=f"{df.iloc[change['new']]['name']} - {plot_columns[0]}",
            xaxis_title="Step",
            yaxis_title="Value",
            height=600
        )
        
        fig.show()

# Set up observer
run_dropdown.observe(update_plot, names='value')

# Display widgets
display(run_dropdown)
display(output)

# Initial plot
update_plot({'new': 0})

In [ ]:
hist = df[df["name"] == ""].iloc[0]["history"]

In [ ]:
hist

In [ ]:
hist = pd.DataFrame(hist)

In [ ]:
hist

In [ ]:
hist_loss = hist[["train/global_step", "train/loss"]]

hist_loss["loss_smoothed"] = hist_loss["train/loss"].rolling(window=2).mean()

hist_loss["gradient"] = hist_loss["train/loss"].diff()

px.line(hist_loss, x="train/global_step", y=["train/loss", "loss_smoothed", "gradient"], title="Training Loss with Smoothing")

In [ ]:
# Peak Detection using scipy.signal.find_peaks (STRICT)
from scipy.signal import find_peaks
import numpy as np

# Find peaks in training loss
loss_data = hist_loss["train/loss"].values
steps = hist_loss["train/global_step"].values

# Calculate adaptive thresholds for stricter detection
loss_mean = np.mean(loss_data)
loss_std = np.std(loss_data)
# min_height = loss_mean + 0.5 * loss_std  # Only peaks above mean + 0.5 std

# Find peaks with STRICT parameters
peaks, properties = find_peaks(
    loss_data, 
    # height=min_height,    # Minimum height - only significant peaks
    threshold=None,       # Required threshold of peaks
    distance=10,          # Increased distance between peaks (was 5)
    prominence=1,       # Increased prominence requirement (was 0.01)
    width=2,              # Minimum width of peaks
    wlen=None,            # Window length for prominence calculation
    rel_height=0.5        # Relative height for width calculation
)

# Create plot
fig = px.line(hist_loss, x="train/global_step", y="train/loss", title="Training Loss with Strict Peak Detection")

# # Add horizontal line showing minimum height threshold
# fig.add_hline(y=min_height, line_dash="dash", line_color="gray", 
#               annotation_text=f"Min Height: {min_height:.4f}")

# Add peaks
if len(peaks) > 0:
    peak_steps = steps[peaks]
    peak_values = loss_data[peaks]
    
    fig.add_scatter(
        x=peak_steps,
        y=peak_values,
        mode='markers',
        marker=dict(color='red', size=12, symbol='triangle-up'),
        name=f'Strict Peaks ({len(peaks)})'
    )

fig.show()

# Print peak information
print(f"Found {len(peaks)} strict peaks:")
print(f"Detection criteria:")
# print(f"  - Minimum height: {min_height:.4f}")
print(f"  - Minimum distance: 20 steps")
print(f"  - Minimum prominence: 0.1")
print(f"  - Minimum width: 3 steps")

if len(peaks) > 0:
    for i, peak_idx in enumerate(peaks):
        step = steps[peak_idx]
        value = loss_data[peak_idx]
        prominence = properties['prominences'][i] if 'prominences' in properties else 'N/A'
        width = properties['widths'][i] if 'widths' in properties else 'N/A'
        print(f"  Peak {i+1}: Step {step}, Loss = {value:.4f}, Prominence = {prominence:.4f}, Width = {width:.1f}")
else:
    print("  No strict peaks detected")

In [ ]:
hist["train/loss"].diff().diff().diff().diff().plot()

In [ ]:
def plot_train_loss(history):
    hist = pd.DataFrame(history)
    fig = px.line(hist, y="eval/loss", title="Training Loss Over Time")
    return fig
    

In [ ]:
plot_train_loss(df.iloc[3]["history"])

In [ ]:
# Let's examine the structure of the data
print("DataFrame shape:", df.shape)
print("DataFrame columns:", df.columns.tolist())
print("\nFirst run info:")
print("Name:", df.iloc[0]['name'])
print("History type:", type(df.iloc[0]['history']))
print("History sample:", str(df.iloc[0]['history'])[:200] + "..." if len(str(df.iloc[0]['history'])) > 200 else str(df.iloc[0]['history']))

In [ ]:
# Let's debug the metric detection issue
import ast

def debug_parse_history(history_str):
    """Debug version of parse_history"""
    try:
        if isinstance(history_str, str):
            history_list = ast.literal_eval(history_str)
        else:
            history_list = history_str
        
        if history_list and len(history_list) > 0:
            hist_df = pd.DataFrame(history_list)
            print(f"Parsed DataFrame shape: {hist_df.shape}")
            print(f"Columns: {hist_df.columns.tolist()}")
            print(f"Numeric columns: {hist_df.select_dtypes(include=['float64', 'int64']).columns.tolist()}")
            return hist_df
        else:
            print("Empty history list")
            return pd.DataFrame()
    except Exception as e:
        print(f"Error parsing history: {e}")
        return pd.DataFrame()

# Test with first run
print("=== Debugging first run ===")
first_run = df.iloc[0]
print(f"Run name: {first_run['name']}")
debug_hist = debug_parse_history(first_run['history'])

In [ ]:
# Let's see what's actually in the history string
history_sample = df.iloc[0]['history']
print("History type:", type(history_sample))
print("First 500 characters:")
print(repr(history_sample[:500]))
print("\nLast 200 characters:")
print(repr(history_sample[-200:]))

In [ ]:
# Test the fixed parsing function
def parse_history_fixed(history_str):
    """Parse history string into DataFrame"""
    try:
        if isinstance(history_str, str):
            # Replace 'nan' with 'None' to make it parseable
            cleaned_str = history_str.replace('nan', 'None')
            history_list = ast.literal_eval(cleaned_str)
        else:
            history_list = history_str
        
        if history_list and len(history_list) > 0:
            df_hist = pd.DataFrame(history_list)
            # Convert None back to NaN
            df_hist = df_hist.replace({None: np.nan})
            return df_hist
        else:
            return pd.DataFrame()
    except Exception:
        # If ast.literal_eval fails, try eval as a last resort
        try:
            if isinstance(history_str, str):
                # Create a safe environment for eval
                safe_dict = {"nan": float('nan'), "__builtins__": {}}
                history_list = eval(history_str, safe_dict)
                if history_list and len(history_list) > 0:
                    return pd.DataFrame(history_list)
            return pd.DataFrame()
        except Exception:
            return pd.DataFrame()

# Test with the first run
print("Testing fixed parsing...")
test_hist = parse_history_fixed(df.iloc[0]['history'])
print(f"Success! Shape: {test_hist.shape}")
print(f"Columns: {test_hist.columns.tolist()}")
print(f"Numeric columns: {test_hist.select_dtypes(include=[np.number]).columns.tolist()}")

In [ ]:
import numpy as np

# Test getting all available metrics
def get_available_metrics_fixed(df):
    """Get all available metrics from all runs"""
    all_metrics = set()
    
    for idx, row in df.iterrows():
        hist_df = parse_history_fixed(row['history'])
        if not hist_df.empty:
            # Get numeric columns (exclude timestamp and step columns for metric selection)
            numeric_cols = hist_df.select_dtypes(include=[np.number]).columns.tolist()
            all_metrics.update(numeric_cols)
    
    # Remove common non-metric columns
    exclude_cols = {'_timestamp', '_runtime', '_step'}
    all_metrics = all_metrics - exclude_cols
    
    return sorted(list(all_metrics))

print("Testing metric detection...")
available_metrics = get_available_metrics_fixed(df)
print(f"Found {len(available_metrics)} metrics:")
for metric in available_metrics:
    print(f"  - {metric}")

In [ ]:
# Test the trimming functionality
sample_data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print("Original data:", sample_data)

# Test percentage trimming (20%)
skip_percentage = 20
skip_count = int(len(sample_data) * skip_percentage / 100)
trimmed_data = sample_data[skip_count:]
print(f"After {skip_percentage}% trimming (skip {skip_count} rows):", trimmed_data)

# Test fixed trimming (3 rows)
skip_rows = 3
trimmed_data_fixed = sample_data[skip_rows:]
print(f"After skipping {skip_rows} rows:", trimmed_data_fixed)

print("\n✅ Trimming logic works correctly!")